In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [17]:
movies = pd.read_csv('/movies.csv')
ratings = pd.read_csv('/ratings.csv')
ratings = pd.merge(movies,ratings).drop(['timestamp','genres'],axis=1)
ratings.head

<bound method NDFrame.head of         movieId                                      title  userId  rating
0             1                           Toy Story (1995)       1     4.0
1             1                           Toy Story (1995)       5     4.0
2             1                           Toy Story (1995)       7     4.5
3             1                           Toy Story (1995)      15     2.5
4             1                           Toy Story (1995)      17     4.5
...         ...                                        ...     ...     ...
100831   193581  Black Butler: Book of the Atlantic (2017)     184     4.0
100832   193583               No Game No Life: Zero (2017)     184     3.5
100833   193585                               Flint (2017)     184     3.5
100834   193587        Bungo Stray Dogs: Dead Apple (2018)     184     3.5
100835   193609        Andrew Dice Clay: Dice Rules (1991)     331     4.0

[100836 rows x 4 columns]>

In [23]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings = userRatings.dropna(thresh=10,axis=1).fillna(0,axis=1)
userRatings.head

<bound method NDFrame.head of title   'burbs, The (1989)  (500) Days of Summer (2009)  \
userId                                                    
1                      0.0                          0.0   
2                      0.0                          0.0   
3                      0.0                          0.0   
4                      0.0                          0.0   
5                      0.0                          0.0   
...                    ...                          ...   
606                    0.0                          0.0   
607                    0.0                          0.0   
608                    0.0                          0.0   
609                    0.0                          0.0   
610                    0.0                          3.5   

title   10 Cloverfield Lane (2016)  10 Things I Hate About You (1999)  \
userId                                                                  
1                              0.0                      

In [24]:
from scipy import sparse

In [27]:
corr_rating_matrix = userRatings.corr('pearson')
corr_rating_matrix.head

<bound method NDFrame.head of title                              'burbs, The (1989)  \
title                                                   
'burbs, The (1989)                           1.000000   
(500) Days of Summer (2009)                  0.063117   
10 Cloverfield Lane (2016)                  -0.023768   
10 Things I Hate About You (1999)            0.143482   
10,000 BC (2008)                             0.011998   
...                                               ...   
Zoolander (2001)                             0.049897   
Zootopia (2016)                              0.003233   
eXistenZ (1999)                              0.187953   
xXx (2002)                                   0.062174   
¡Three Amigos! (1986)                        0.353194   

title                              (500) Days of Summer (2009)  \
title                                                            
'burbs, The (1989)                                    0.063117   
(500) Days of Summer (2009)   

In [38]:
def get_similar_mvis(mvi,rating):
  similar_mvis = corr_rating_matrix[mvi] * (rating-2.5)
  similar_mvis.sort_values(ascending=False)
  return similar_mvis

In [52]:
user_x = [("Zootopia (2016)",5)]
similar_mvi=pd.DataFrame()
for mvi,rating in user_x:
  similar_mvi=similar_mvi.append(get_similar_mvis(mvi,rating))
similar_mvi.head(10)

,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
Zootopia (2016),0.008082,0.540017,0.798428,0.125079,0.236933,0.135061,0.193986,0.141854,0.158314,0.564368,...,0.715534,0.391508,0.028545,0.535963,0.746261,0.270368,2.5,0.117213,0.501904,0.051488


In [54]:
similar_mvi.sum().sort_values(ascending=False).head(10)

Zootopia (2016)                                      2.500000
Inside Out (2015)                                    1.442461
Deadpool (2016)                                      1.385494
Big Hero 6 (2014)                                    1.372813
Star Wars: Episode VII - The Force Awakens (2015)    1.348236
Guardians of the Galaxy (2014)                       1.335005
Doctor Strange (2016)                                1.287813
Mad Max: Fury Road (2015)                            1.266820
Kingsman: The Secret Service (2015)                  1.217669
The Lego Movie (2014)                                1.212119
dtype: float64